# SAP Cloud ALM Project and Task API Demo for Requirements

This notebook contains examples of SAP Business Hub API calls for SAP Cloud ALM for Implementation. The code cells below show how to use the Project and Task APIs to read and update Requirements.

The API information and specification is available here:

* https://api.sap.com/package/SAPCloudALM/rest - SAP Cloud ALM
* https://api.sap.com/api/CALM_PJM/overview - SAP Cloud ALM Projects
* https://api.sap.com/api/CALM_TKM/overview - SAP Cloud ALM Tasks

Please note the license and other terms and conditions contained in this notebook's repository: https://github.com/SAP-samples/cloud-alm-api-examples

## Python Dependencies Required

In order to run the samples in this notebook, install the following dependencies:

* Jupyter integration in Visual Studio Code: https://code.visualstudio.com/docs/python/jupyter-support 
* Python 3, a recent version, is Required. Python 3.8 was used here.
* Requests - for handling HTTP GET/POST/PATCH/DELETE Requests - https://docs.python-requests.org/en/latest/user/install/#install
* Requests-OAuthlib - for authentication with requests - https://requests-oauthlib.readthedocs.io/en/latest/index.html#installation
* Pandas - Python data analysis - https://pandas.pydata.org/docs/getting_started/install.html
* Plotly - for plotting interactive charts - https://plotly.com/python/getting-started/

## APIs called

API to get projects: https://<tenant url\>/api/imp-pjm-srv/v1/projects

API to get tasks, including Requirements: https://<tenant url\>/api/imp-tkm-srv/v1/tasks?projectId=<project ID\>

## Requirement Specifics

Requirements are treated exactly like tasks, and are accessed through the task API. There are several additional attributes relevant for requirements.

* Requirement task type = `CALMREQU`
* `subStatus` attribute for the Requirement Status
* `approvalState` = `APPROVED` if the requirement has been approved

Requirements have both a status, and corresponding to each status value, a subStatus.

* `CREATED` = In Specification, main status = CIPREQUOPEN
* `IN_REALIZATION` = In Realization, main status =  CIPREQUINP
* `APPROVED_FOR_DEPLOYMENT` = In Deployment, main status = CIPREQUINP
* `CONFIRMED` = Deployed, main status = CIPREQUCLOSE
* `NOT_PLANNED` = Not Planned, main status = CIPREQUNO

These values will be extended


---

## Authentication information

You must create a python module file called `apidata.py` and put the information specific to your tenant there. This inclduds

* OAuth2 client ID and secret
* Token url
* Base URL for API calls

Get client ID and secret variables from an external module: this information is senstive.

These items can be retrieved from the BTP Cockpit 

### Format of module apidata.py for import

```python
ptm_all_client_id = r'get your client ID from BTP Cockpit'
ptm_all_client_secret = r'get your client secret from BTP Cockpit'
token_url = 'your token url'
base_url = 'your base url'
```


In [ ]:
import apidata as ad

client_id = ad.ptm_all_client_id
client_secret = ad.ptm_all_client_secret
token_url = ad.token_url
base_url = ad.base_url

### Get token for authentication

Call OAuth token API with credential information. Add the resulting header to all requests.

See Requests-OAuthlib documentation for Backend Application Flow:

* https://requests-oauthlib.readthedocs.io/en/latest/oauth2_workflow.html#backend-application-flow

In [ ]:
import requests
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient

client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url=token_url, client_id=client_id,
        client_secret=client_secret)

hed = {'Authorization': 'Bearer ' + token['access_token']}        


---

## Perform GET request to retrieve list of all projects

Expected response: "200 OK"

In [ ]:
response = requests.get(base_url + '/api/calm-projects/v1/projects', headers=hed)

print(response.status_code, response.reason)

## Parse JSON into Pandas Dataframe

This takes the data returned from the projects API, which is in JSON format, and places it into a dataframe for further processing and analysis.

In [ ]:
import pandas as pd

df = pd.json_normalize(response.json())

df

## Get a Project Name 

You may wish to adjust the code below to set a project name of your choosing, for example 'My API Test Project'. In this sample, we'll just take the first project in the list.


In [ ]:
sample_project_name = df.iloc[0]['name'] # First project in the list
sample_project_name

### Override Sample Project Name for Demo

I'm setting my own project name here, beause I have one ready for my demos. You can do the same if you don't want to use the first project

In [ ]:
# Uncomment and change to the name of a project you would like to use:

#sample_project_name = 'API Demo Project'

## Get ID of a Selected Project

Read ID of project 'Show & Tell 2021-Apr-30' from dataframe

In [ ]:
stPrjID = df.loc[df['name'] == sample_project_name]
show_tell_prj = stPrjID['id'].values[0]
print('Project:', stPrjID['name'].values[0], 'ID:', show_tell_prj)


---

## Read Task list from selected project

Use GET request from tasks API with selected project ID as input parameter.

Expected response "200 OK"

In [ ]:
taskURL = base_url + '/api/calm-tasks/v1/tasks?projectId=' + show_tell_prj

response = requests.get(taskURL, headers=hed)

print(response.status_code, response.reason)

### Parse project task list JSON into dataframe

In [ ]:
df_stTasks = pd.json_normalize(response.json())

df_stTasks.head()

In [ ]:
df_stTasks

### Barchart plot of tasks by type

Use built-in matplotlib of Pandas for quick data exploration

Requirements have type `CALMREQU`.

In [ ]:
df_stTasks.type.value_counts().plot.pie(title="Task Type", figsize=(18,8))

### Analyze Requirements by substatus, approvalState

Count Requirements in each substatus (called "Requirement Status")

Count approved Requirements

In [ ]:
df_stTasks.groupby(by=['subStatus'])['id'].count().reset_index()

In [ ]:
df_stTasks.subStatus.value_counts().plot.pie(title="Requirement Status", figsize=(18,8))

In [ ]:
df_stTasks.groupby(by=['approvalState'])['id'].count().reset_index()


---

## Create Requirement

* Endpoint: `https://<tenant url>/api/imp-tkm-srv/v1/tasks`

* Type: Post

Append a timestamp to the end of the new task title for easy identification in UI.

Expected response `201 Created`


In [ ]:
import datetime

isoNow = datetime.datetime.now().isoformat()

taskCreateURL = base_url + '/api/calm-tasks/v1/tasks'

taskData = {
    "projectId": show_tell_prj,
    "title": "Notebook API test Requirement " + isoNow,
    "type": "CALMREQU",
    "description": "test description. hello world",
    "assigneeId": "test.user@example.com"
}

response = requests.post(taskURL, headers=hed, json=taskData)

print(response.status_code, response.reason)

newTaskID = response.json()['id']
print("New Requirement ID:", newTaskID)


---

## Get Requirement Details

Use GET request to read details of requirement just created

* Endpoint: `https://<tenant url>/rest/tkm/v1/tasks/{taskGuid}`

* Type: Get

Expected response: `200 OK`


In [ ]:
taskURL = base_url + '/api/calm-tasks/v1/tasks/' + newTaskID

response = requests.get(taskURL, headers=hed)

print(response.status_code, response.reason)

### Save and print response JSON

In [ ]:
readTask = response.json()
readTask


---

## Modify Requirement

Use PATCH method to update Requirement attributes

Set the main status to `CIPREQUINP`

Expected response: `200 OK`

In [ ]:
taskToModify = {}

taskToModify['title'] = readTask['title'] + ' mod'
taskToModify['dueDate'] = '2022-04-09'
taskToModify['status'] = 'CIPREQUINP'
taskToModify['description'] = 'This is a new <b>Description</b>. It replaces the old description'

### Set subStatus, approvalState

Set the `subStatus` and `approvalState` attributes, which are the Requirement specific attributes

Set a corresponding subStatus for `CIPREQUINP` to `TO_BE_APPROVED` - "In Approval"

Set the approvalState to `READY_4_APPR` - "Ready for Approval"

See the schema definition in the SAP API Business Hub for allowed status, sub status, and approval state values. 

* https://api.sap.com/api/CALM_TKM/schema

In [ ]:
taskToModify['subStatus'] = 'TO_BE_APPROVED'
taskToModify['approvalState'] = 'READY_4_APPR'

In [ ]:
taskURL = base_url + '/api/calm-tasks/v1/tasks/' + newTaskID

response = requests.patch(taskURL, headers=hed, json=taskToModify)

print(response.status_code, response.reason)

In [ ]:
response.json()


---

## Delete Requirement

DELETE method - the system **forbids** deletion of **approved** Requirements

Expected response: `204 No Content` if the Requirement is not approved, `400 Bad Request` if the Requirement is approved.


In [ ]:
taskURL = base_url + '/api/calm-tasks/v1/tasks/' + newTaskID

# Uncomment code below to test deletion

#response = requests.delete(taskURL, headers=hed)

#print(response.status_code, response.reason)